# Traduire le langage en probabilités

Quelle est la probabilité de tirer la lettre *A* au *Scrabble* ? La question, anodine, ne peut se résoudre que si nous connaissons deux quantités :

- le nombre total de jetons ;
- le nombre de jetons *A*.

Quelle est maintenant la probabilité de réalisation de l’événement *i* dans le verbe *boire* ? La réponse est le quotient entre le nombre de lettres *i* dans le verbe donné avec le nombre total de lettres qui le composent.

## La fréquence relative

Dénombrer les lettres d’un texte ne donne qu’une mesure absolue de la présence de chacune, sans ne rien dire de leur importance. Qu’une lettre apparaisse trois mille fois est en soi beaucoup, mais au milieu d’un corpus de trois milliards de mots, elle ne pèse guère, d’où la nécessité de toujours considérer un chiffre dans son contexte.

Plus formellement, la relation précédente s’exprime par l’équation :

$$
P(c) = \frac{F(c)}{N}
$$

Dans la formule, $P(c)$ est la probabilité de réalisation de l’événement $c$ – à savoir un caractère, $F(c)$ la fréquence d’apparition du caractère et $N$ la taille du corpus. En l’appliquant au verbe plus haut, on peut en déduire que la probabilité de l’événement *i* est égale à :

$$
P(i) = \frac{1}{5} = 0,2
$$

Dans ce contexte, la réalisation d’un événement aléatoire est jugée indépendante des autres événements. Ainsi, la probabilité de réaliser l’événement *i* puis l’événement *r*, soit d’obtenir exactement la séquence *ir* revient à multiplier les probabilités des deux événements :

$$
P(i \cap r) = P(i) \times P(r) = 0,2 \times 0,2 = 0,04
$$

## Un modèle *n*-grammes

Considérons la phrase suivante :

>Le petit chat boit du lait.

Quelle serait maintenant la probabilité que l’événement *it* se produise ? La question est différente de la précédente comme on considère l’ensemble *it* comme un événement et plus comme deux événements distincts. Il est par conséquent possible d’appliquer la première formule :

$$
P(it) = \frac{F(it)}{N-1}
$$

Pourquoi $N-1$ ? Les fréquences calculées précédemment n’impliquaient que des unigrammes (ou 1-grammes) or, ici, nous souhaitons estimer la probabilité d’un bigramme (ou 2-grammes). La règle est linéaire, de telle manière qu’il existe $N-2$ trigrammes, $N-3$ tétragrammes, $N-4$ 5-grammes etc. Considérant qu’il y a 27 caractères dans la phrase, espaces et ponctuation comprises, la probabilité de l’événement *it* s’établit ainsi à :

$$
P(it) = \frac{3}{27 - 1} \approx 0.1154
$$

Dans un modèle *n*-grammes toutefois, la question n’est pas réellement d’estimer la probabilité d’un événement mais plutôt sa vraisemblance en fonction d’un historique. Reformulons : quelle serait la probabilité de l’événement *t* sachant que *i* est arrivé juste avant ? Du point de vue mathématique, la formule revient à :

$$
P(t|i) = \frac{F(it)}{F(i)} = \frac{3}{3} = 1
$$

En effet, dans le corpus, un *i* est toujours suivi d’un *t*.

## Des contraintes pesant sur les modèles

### Formule des probabilités composées

L’intuition première pour calculer des probabilités sur des chaînes de caractères serait d’appliquer une chaîne de traitement sans historicité :

$$
P(u_1 u_2 \ldots u_n) \approx \prod_i P(u_i)
$$

Pour obtenir le mot *chai*, il suffirait ainsi de multiplier entre elles les probabilités de chacune des lettres :

$$
\begin{aligned}
P(chai) &= P(c) \cdot P(h) \cdot P(a) \cdot P(i)\\
&= \frac{1}{27} \cdot \frac{1}{27} \cdot \frac{2}{27} \cdot \frac{3}{27} \approx 1,129 \times 10^{-5}
\end{aligned}
$$

Et dans un modèle bigramme, la logique reste la même, chaque unité étant cette fois-ci composée de deux caractères :

$$
P(chai) = P(ch) \cdot P(ha) \cdot P(ai) = \frac{1}{26} \cdot \frac{1}{26} \cdot \frac{1}{26} = 0,0000569
$$

Ce genre de modèle qui ne tient pas compte du contexte reste très limité : un générateur basé sur une telle règle écrirait des caractères aléatoirement. Après tout, les chaînes *du* et *hb* bénéficient de la même estimation de probabilité alors que le bigramme *du* existe dans le corpus mais pas *hb* :

$$
P(du) = P(d) * P(u) \approx 0.0384 \cdot 0.0384 \approx 0.001479
$$

$$
P(hb) = P(h) * P(b) \approx 0.0384 \cdot 0.0384 \approx 0.001479
$$

Une meilleure approche consiste à poser le problème sous forme de probabilités conditionnelles :

$$
P(chai) = P(h|c) \cdot P(a|ch) \cdot P(i|cha)
$$

Une formule qui peut se généraliser :

$$
P(u_1 u_2 \ldots u_n) = \prod_{i=1} P(u_i ∣ u_0 \ldots u_{i−1})
$$

Si elle semble prometteuse, sur l’argument qu’elle repose toujours sur l’antériorité, elle aboutit très rapidement à une probabilité de 0 pour des énoncés pourtant probables :

$$
\begin{aligned}
P(chai) &= P(h|c) \cdot P(a|ch) \cdot P(i|cha)\\
&= \frac{F(ch)}{F(c)} \cdot \frac{F(cha)}{F(ch)} \cdot \frac{F(chai)}{F(cha)}\\
&= \frac{1}{1} \cdot \frac{1}{1} \cdot \frac{0}{1}\\
&= 1 \cdot 1 \cdot 0\\
&= 0
\end{aligned}
$$

### Hypothèses de correction

Pour éviter cet écueil, la première intuition serait d’ignorer tout simplement les cas qui n’apparaissent pas dans le corpus d’entraînement. Si nous retenons cette idée, $P(chai)$ devient subitement égale à 1. Passer de 0 à 100 % de chances de voir le mot *chai* (autant que le mot *chat*) semble indiquer que notre modèle de langage n’est pas très rationnel.

Une correction plus envisageable, et qui prend le nom de **lissage de Laplace**, serait d’augmenter de 1 toutes les fréquences du corpus et d’assigner la même valeur aux évènements nuls :

$$
\begin{aligned}
P(chat) &= \frac{F(ch)+1}{F(c)+1} \cdot \frac{F(cha)+1}{F(ch)+1} \cdot \frac{F(chat)+1}{F(cha)+1}\\
&= \frac{2}{2} \cdot \frac{2}{2} \cdot \frac{2}{2}\\
&= 1\\
P(chai) &= \frac{F(ch)+1}{F(c)+1} \cdot \frac{F(cha)+1}{F(ch)+1} \cdot \frac{F(chai)+1}{F(cha)+1}\\
&= \frac{2}{2} \cdot \frac{2}{2} \cdot \frac{1}{2}\\
&= 0.5
\end{aligned}
$$

En poursuivant cette logique, une meilleure approximation de la probabilité d’un évènement inconnu s’appuierait sur une pondération du maximum de vraisemblance. Fixons le paramètre $\alpha$ à 0,05 pour la probabilité d’un évènement inconnu et un paramètre $k$ pour la pondération qui vaut $1 - \alpha$ soit 0,95 :

$$
P(chai) = 0.95 \times 0 + \frac{0.05}{27} \approx 0.0019
$$

La formule vaut ainsi :

$$
P(w_i) = k \cdot P_{ML}(w_i) + \frac{\alpha}{N}
$$

Dans la réalité, le paramètre $k$ est ajusté sur le corpus d’entraînement plutôt que déterminé arbitrairement. D’autres techniques existent, comme les actualisations de Good-Turing et de Witten-Bell, ou encore le lissage de Kneser-Ney qui estiment les évènements inconnus sur la base des hapax.

### L’hypothèse markovienne

En théorie, la formule des probabilités composées couvre bien les besoins impliqués par la construction d’un modèle de langage simple. Après tout, quand un corpus d’apprentissage n’est constitué que de quelques caractères et que le nombre de caractères d’un mot est fini, il semble envisageable de calculer toutes les possibilités.

Étendons notre exemple au français, qui dispose d’un alphabet de 26 lettres et dont le mot le plus long, *anticonstitutionnellement*, en compte 25. Là encore, générer un mot de *n* caractères en partant d’un état initial et en se fondant sur des données d’entraînement suffisamment importantes semble à portée de main. Oui, mais… qu’en est-il des mots composés ? Et dans les autres langues ? Les règles de composition de l’allemand font par exemple le bonheur des écoliers qui inventent les péripéties d’un capitaine naviguant sur le Danube et le Rhin (*Donaurheinschifffahrtskapitän*) afin d’égaler la prouesse établie par la *Donau­dampf­schiffahrts­elektrizitäten­haupt­betriebs­werk­bau­unter­beamten­gesellschaft*.

Rappelons aussi que l’unité de base des modèles de langage n’est pas le caractère mais le mot. L’alphabet devient lexique et la longueur d’une phrase étant virtuellement infinie, les besoins en calcul sont si délirants qu’il ne sera jamais possible d’estimer de telles probabilités.

Comment faire alors ? C’est ici qu’intervient l’hypothèse markovienne, appelée ainsi après le mathématicien russe Andreï Markov, qui suggère que la probabilité conditionnelle d’un mot sachant son historique est approximée par la probabilité de ce mot sachant uniquement celui qui le précède. Par exemple, la probabilité que le mot *flots* termine la phrase *Le soleil brille au-dessus des* est proche de celle que le mot *flots* suive directement l’article *des* :

$$
P(\text{flots}|\text{Le soleil brille au-dessus des}) \approx P(\text{flots}|\text{des})
$$

On parle aussi d’**horizon limité**, une approximation qui peut s’écrire sous la forme :

$$
P(w_{n+1}|w_1 \ldots w_n) \approx P(w_{n+1}|w_n)
$$

Mieux, l’hypothèse markovienne dans sa généralisation définit une fenêtre contextuelle à gauche :

$$
P(w_1 w_2 \ldots w_n) \approx \prod_i P(w_i|w_{i-k} \ldots w_{i-1})
$$

Les modèles probabilistes peuvent s’étendre ainsi aux trigrammes, aux tétragrammes, aux… sauf que le langage fait montre de tant de diversité que les modèles *n*-grammes ne suffisent généralement pas. Prenons un énoncé :

> Le film, encensé par la critique de nombreux médias spécialisés, comme *Les inrockuptibles*, *Télérama*, *ÉcranLarge* ou encore *Première*, qui s’était déjà exprimée favorablement sur l’opus précédent, est fort mauvais.

La structure des phrases, notamment à l’écrit, implique souvent une dépendance longue distance (*long distance dependancy* ou LDD) facilitée par des incises, des relatives ou des tournures interrogatives.

## Étiquetage et décision

Reprenons un exemple bien connu d’étiquetage d’un énoncé ambigu :

```mermaid
flowchart LR
    A("La
    DET"):::academic-->B("petite
    N"):::formal
    B-->C("brise
    V"):::formal
    C-->D("la
    DET"):::formal
    D-->E("glace
    N"):::formal
    E-->F(".
    PONCT"):::academic
    A-->G("petite
    ADJ"):::academic
    G-->H("brise
    N"):::academic
    H-->I("la
    PRO"):::academic
    I-->J("glace
    V"):::academic
    J-->F
    classDef formal fill:#fff,stroke:#D68738,color:#D68738
    classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

Ces deux voies ne sont pas équiprobables et l’interprétation du mot *petite* dépendra fortement du corpus d’apprentissage.

### Analyse d’un unigramme

Pour un mot *w* donné, on peut déterminer son étiquette *t* grâce à sa fréquence d’apparition dans le corpus :

$$
P(t|w) = \frac{F(w,t)}{F(w)}
$$

Si le mot *petite* apparaît 38 fois dont 12 en tant que nom, la probabilité de le voir avec l’étiquette *ADJ* est de :

$$
P(ADJ|petite) = \frac{26}{38} = 0,6842
$$

À noter que la probabilité de *t* sachant *w* peut s’écrire comme un rapport entre la probabilité du couple mot/étiquette et la probabilité du mot :

$$
P(t|w) = \frac{P(t,w)}{P(w)}
$$

Dans notre exemple fictif, imaginons que nous avons entraîné notre étiqueteur sur un corpus de 1000 mots. La formule donnerait :

$$
P(ADJ|petite) = \frac{\frac{26}{1000}}{\frac{38}{1000}} = 0,6842
$$

De manière réciproque, il serait possible de déduire le mot sachant l’étiquette. Toutefois, la précision dans ce sens sera bien moindre : quand on hésite seulement entre quelques étiquettes pour un mot, la quantité de mots possibles pour une étiquette est bien plus importante.

Ce modèle est jugé simpliste : non seulement il choisit parmi plusieurs possibilités la plus probable ($P(ADJ|petite) > P(N|petite)$) mais en plus il ne tient pas compte de l’antériorité. Au mot suivant, *brise*, il pourrait choisir d’attribuer l’étiquette *V*.

### Étiquetage *n*-gramme

#### Définition de la tâche

L’idée sous-jacente des étiqueteurs *n*-gramme est que l’étiquette d’un mot dépend des *n-1* mots qui le précèdent. Dans la pratique, comme les ressources nécessaires pour le calcul des probabilités croissent avec *n* et que la fiabilité des résultats diminue, notamment en raison d’un phénomène de dilution, on se contente d’une faible valeur de *n*.

Trois hypothèses sont retenues :

- l’approche bayésienne  
$P(t|w) = \frac{P(w|t) \cdot P(t)}{P(w)}$
- l’horizon limité de Markov  
$P(t_n|t_{1,n-1}) = P(t_n|t_{n-1})$
- la probablité d’apparition d’un mot pour une étiquette ne dépend que de son étiquette  
$P(w_i|t_{1,n}) = P(w_i|t_i)$

#### Paramètres de résolution

Soient deux ensembles *W* et *T* pour les mots et les étiquettes. À une séquence de *n* mots est associée une séquence de *n* étiquettes de telle manière que :

$$
\begin{aligned}
  w_{1,n} &= w_1 w_2 \ldots w_n\\
  t_{1,n} &= t_1 t_2 \ldots t_n
\end{aligned}
$$

L’objectif, sachant $w_{1,n}$, est de déterminer $t_{1,n}$ en maximisant $P(t_{1,n}|w_{1,n})$, c’est-à-dire, après avoir calculé toutes les probabilités possibles, de ne conserver que la plus forte :

$$
t_{1,n} = max_{t_{1,n}} P(t_{1,n}|w_{1,n})
$$

Ce qui, d’après le théorème de Bayes, s’exprime en renversant la condition :

$$
\begin{aligned}
  P(t_{1,n}|w_{1,n}) &= \frac{P(w_{1,n}|t_{1,n}) \cdot P(t_{1,n})}{P(w_{1,n})}\\
  t_{1,n} &= \text{max} P(w_{1,n}|t_{1,n}) \cdot P(t_{1,n})
\end{aligned}
$$

Il reste deux probabilités à calculer pour lesquelles on applique les hypothèses retenues plus haut :

- $P(w_{1,n}|t_{1,n})$ : hypothèse selon laquelle un mot ne dépend que de son étiquette
- $P(t_{1,n})$ : hypothèse selon laquelle une étiquette dépend de ses *n* étiquettes précédentes

$$
\begin{aligned}
  P(w_{1,n}|t_{1,n}) &= P(w_1|t_1) \cdot P(w_2|t_2) \ldots P(w_n|t_n)\\
  P(t_{1,n}) &= P(t_1) \cdot P(t_2|t_1) \cdot P(t_3|t_2) \ldots P(t_n|t_{n−1})
\end{aligned}
$$

### Modèle de Markov caché d’ordre *N*

Le modèle décrit plus haut est appelé modèle de Markov caché d’ordre 1 (*Hidden Markov Model*, ou HMM) dans la mesure où la probabilité d’apparition d’une étiquette ne dépend que de la précédente. Pour un ordre plus élevé, par exemple pour un HMM d’ordre 2, nous aurons :

$$
P(t_{1,n}) = P(t_1) \cdot P(t_2|t_1) \cdot P(t_3|t_1 t_2) \ldots P(t_n|t_{n−2}t_{n−1})
$$

En appliquant un HMM, la tâche pour un étiquetage *n*-gramme se formulerait ainsi :

$$
t_{1,n} = \text{max} P(w_1|t_1) \cdot \prod_{i=1,n} P(t_i|t_{i-1}) \cdot P(w_i|t_i)
$$

## Étude de cas : entraînement d’un HMM d’ordre 1

Nous souhaitons calculer la probabilité de génération des trois phrases suivantes :

> (A) Le chat mange la souris.  
> (B) La souris mange le chat.  
> (C) La souris mange.

Pour résoudre la tâche, nous souhaitons utiliser un modèle de Markov à états cachés d’ordre 1.

### Définition du corpus d’apprentissage

Considérons un corpus étiqueté de cinq phrases où les étiquettes en parties du discours constituent les états cachés du modèle de Markov :

> (1) Le/DET chat/NOM mange/VER une/DET souris/NOM ./PONCT  
> (2) La/DET fille/NOM lit/VER un/DET livre/NOM ./PONCT  
> (3) Un/DET chien/NOM court/VER dans/PREP le/DET jardin/NOM ./PONCT  
> (4) Nous/PRON mangeons/VER des/DET fruits/NOM frais/ADJ ./PONCT  
> (5) Le/DET petit/ADJ garçon/NOM joue/VER au/PREP+DET ballon/NOM ./PONCT

### Étape 1 : calculer les probabilités d’émission des étiquettes

La première étape consiste à calculer les probabilités d’émission pour chaque mot sachant son étiquette. Par exemple, il y a huit déterminants dans le corpus, dont trois *le*, soit :

$$
P(\text{le}|\text{DET}) = \frac{3}{8} = 0,375
$$

Du point de vue mathématique, on peut généraliser la relation avec la formule :

$$
P(w_i|t_i) = \frac{F(w_i, t_i)}{F(t_i)}
$$

La tableau ci-dessous reprend toutes les probabilités d’émission :

|$w, t$   | DET  | NOM  | VER  | PONCT | PREP | PRON | ADJ  | PREP+DET |
|---------|------|------|------|-------|------|------|------|---------|
| **le**  | 0.375 | 0    | 0    | 0     | 0    | 0    | 0    | 0       |
| **une** | 0.125 | 0    | 0    | 0     | 0    | 0    | 0    | 0       |
| **la**  | 0.125 | 0    | 0    | 0     | 0    | 0    | 0    | 0       |
| **un**  | 0.250 | 0    | 0    | 0     | 0    | 0    | 0    | 0       |
| **des** | 0.125 | 0    | 0    | 0     | 0    | 0    | 0    | 0       |
| **chat** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **souris** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **fille** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **livre** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **chien** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **jardin** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **fruits** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **garçon** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **ballon** | 0    | 0.111 | 0    | 0     | 0    | 0    | 0    | 0       |
| **mange** | 0    | 0    | 0.200 | 0     | 0    | 0    | 0    | 0       |
| **lit** | 0    | 0    | 0.200 | 0     | 0    | 0    | 0    | 0       |
| **court** | 0    | 0    | 0.200 | 0     | 0    | 0    | 0    | 0       |
| **mangeons** | 0    | 0    | 0.200 | 0     | 0    | 0    | 0    | 0       |
| **joue** | 0    | 0    | 0.200 | 0     | 0    | 0    | 0    | 0       |
| **.** | 0    | 0    | 0    | 1.000 | 0    | 0    | 0    | 0       |
| **dans** | 0    | 0    | 0    | 0     | 1.000 | 0    | 0    | 0       |
| **nous** | 0    | 0    | 0    | 0     | 0    | 1.000 | 0    | 0       |
| **frais** | 0    | 0    | 0    | 0     | 0    | 0    | 0.500 | 0       |
| **petit** | 0    | 0    | 0    | 0     | 0    | 0    | 0.500 | 0       |
| **au** | 0    | 0    | 0    | 0     | 0    | 0    | 0    | 1.000   |

### Étape 2 : calculer les probabilités de transition

Un HMM d’ordre 1 n’estime la probabilité d’apparition d’une étiquette qu’en fonction de celle qui la précède directement :

$$
P(t_i|t_{i-1}) = \frac{F(t_{i-1} t_i)}{F(t_{i-1})}
$$

Cette relation nous permet de calculer les probabilités de transition d’un état à l’autre. Par exemple, sur huit occurrences, un déterminant est suivi sept fois par un nom contre une seule fois par un adjectif. Soit :

$$
P(\text{NOM}|\text{DET}) = \frac{7}{8} = 0,875
$$

Nous reportons ci-dessous toutes les probabilités de transition :

|$t_i, t_{i+1}$  | START | DET  | NOM  | VER  | PONCT | PREP | PRON | ADJ  | PREP+DET |
|---------------|-------|------|------|------|-------|------|------|------|---------|
| **START**    | 0     | 0.800 | 0    | 0    | 0     | 0    | 0.200 | 0    | 0       |
| **DET**      | 0     | 0     | 0.875 | 0    | 0     | 0    | 0     | 0.125 | 0       |
| **NOM**      | 0     | 0     | 0    | 0.444 | 0.444 | 0    | 0     | 0.111 | 0       |
| **VER**      | 0     | 0.600 | 0    | 0    | 0     | 0.200 | 0     | 0    | 0.200   |
| **PONCT**    | 0     | 0     | 0    | 0    | 0     | 0    | 0     | 0    | 0       |
| **PREP**     | 0     | 1.000 | 0    | 0    | 0     | 0    | 0     | 0    | 0       |
| **PRON**     | 0     | 0     | 0    | 1.000 | 0     | 0    | 0     | 0    | 0       |
| **ADJ**      | 0     | 0     | 0.500 | 0    | 0.500 | 0    | 0     | 0    | 0       |
| **PREP+DET** | 0     | 0     | 1.000 | 0    | 0     | 0    | 0     | 0    | 0       |

### Étape 3 : établir le modèle

Si nous reprenons la définition d’un modèle de Markov caché d’ordre *N*, nous pouvons formaliser l’expression suivante :

$$
P(w_1, \dots, w_n, t_1, \dots, t_n) = P(t_1 | \text{START}) \cdot P(w_1 | t_1) \cdot \prod_{i=2}^{n} P(t_i | t_{i-1}) \cdot P(w_i | t_i)
$$

### Étape 4 : extraire les probabilités d’émission et de transition

#### Probabilités d’émission

D’après les matrices calculées aux étapes précédentes, nous pouvons extraire les probabilités d’émission des mots de nos phrases en fonction de leurs étiquettes :

- A :
$$
\begin{aligned}
  P(\text{le} ∣ \text{DET}) &= 0,375\\
  P(\text{chat} ∣ \text{NOM}) &= 0,111\\
  P(\text{mange} ∣ \text{VER}) &= 0,200\\
  P(\text{la} ∣ \text{DET}) &= 0,125\\
  P(\text{souris} ∣ \text{NOM}) &= 0,111\\
  P(\text{.} ∣ \text{PONCT}) &= 1
\end{aligned}
$$

- B :
$$
\begin{aligned}
  P(\text{la} ∣ \text{DET}) &= 0,125\\
  P(\text{souris} ∣ \text{NOM}) &= 0,111\\
  P(\text{mange} ∣ \text{VER}) &= 0,200\\
  P(\text{le} ∣ \text{DET}) &= 0,375\\
  P(\text{chat} ∣ \text{NOM}) &= 0,111\\
  P(\text{.} ∣ \text{PONCT}) &= 1
\end{aligned}
$$

- C :
$$
\begin{aligned}
  P(\text{la} ∣ \text{DET}) &= 0,125\\
  P(\text{souris} ∣ \text{NOM}) &= 0,111\\
  P(\text{mange} ∣ \text{VER}) &= 0,200\\
  P(\text{.} ∣ \text{PONCT}) &= 1
\end{aligned}
$$

#### Probabilités de transition

De façon similaire, nous pouvons reporter les probabilités de transition :

- A :
$$
\begin{aligned}
  P(\text{DET}∣\text{START}) &= 0,8\\
  P(\text{NOM}∣\text{DET}) &= 0,875\\
  P(\text{VER}∣\text{NOM}) &= 0,444\\
  P(\text{DET}∣\text{VER}) &= 0,6\\
  P(\text{NOM}∣\text{DET}) &= 0,875\\
  P(\text{PONCT}∣\text{NOM}) &= 0,444
\end{aligned}
$$

- B :
$$
\begin{aligned}
  P(\text{DET}∣\text{START}) &= 0,8\\
  P(\text{NOM}∣\text{DET}) &= 0,875\\
  P(\text{VER}∣\text{NOM}) &= 0,444\\
  P(\text{DET}∣\text{VER}) &= 0,6\\
  P(\text{NOM}∣\text{DET}) &= 0,875\\
  P(\text{PONCT}∣\text{NOM}) &= 0,444
\end{aligned}
$$

- C :
$$
\begin{aligned}
  P(\text{DET}∣\text{START}) &= 0,8\\
  P(\text{NOM}∣\text{DET}) &= 0,875\\
  P(\text{VER}∣\text{NOM}) &= 0,444\\
  P(\text{PONCT}∣\text{VER}) &= 0
\end{aligned}
$$

### Étape 5 : calculer la probabilité des phrases

Ainsi, en appliquant le modèle défini à l’étape 3, on obtient les probabilités suivantes pour les phrases A, B et C :

$$
\begin{aligned}
  P(A) &= 0,8 \times 0,375 \times 0,875 \times 0,111 \times 0,444 \times 0,200 \times 0,600 \times 0,125 \times 0,875 \times 0,111 \times 0,444 \times 1\\
  &\approx 8,37 \times 10^{-6}\\
  P(B) &= 0,8 \times 0,125 \times 0,875 \times 0,111 \times 0,444 \times 0,200 \times 0,600 \times 0,375 \times 0,875 \times 0,111 \times 0,444 \times 1\\
  &\approx 8,37 \times 10^{-6}\\
  P(C) &= 0,8 \times 0,125 \times 0,875 \times 0,111 \times 0,444 \times 0,200 \times 0\\
  &= 0
\end{aligned}
$$

### Conclusion

On observe que les phrases A et B ont exactement la même probabilité, bien que B soit hautement improbable d’un point de vue sémantique (une souris qui mange un chat). Cela montre que le modèle HMM ne prend en compte que les enchaînements de catégories grammaticales et non le sens des phrases.  

À l’inverse, la phrase C, qui est parfaitement naturelle en français, est jugée impossible par le modèle simplement parce que la transition $P(\text{PONCT} | \text{VER})$ est absente du corpus d’apprentissage. Cela illustre une limitation des HMM : si une transition n’a jamais été observée, sa probabilité est nulle, ce qui exclut certaines phrases pourtant valides.  

Un facteur clé dans ces observations est la **dépendance totale du modèle au corpus d’apprentissage**. Un HMM ne peut estimer des probabilités que sur ce qu’il a vu, ce qui pose problème si le corpus est trop restreint ou biaisé. Un corpus déséquilibré peut ainsi fausser les probabilités et entraîner des erreurs dans l’analyse des séquences. Pour améliorer la robustesse du modèle, il est essentiel d’avoir un corpus représentatif et d’utiliser des techniques de lissage. Le lissage de Laplace est une approche simple, mais souvent trop naïve car il attribue la même correction à toutes les probabilités. Des méthodes plus avancées, comme le **lissage additif optimisé**, le **lissage de Good-Turing** ou encore le **lissage de Kneser-Ney**, permettent une estimation plus fine des probabilités en tenant compte de la structure du corpus et des distributions observées.

Les HMM d’ordre 1, comme celui utilisé ici, ne prennent en compte que la catégorie précédente pour prédire l’actuelle. Cette hypothèse simplificatrice empêche le modèle de capturer des **dépendances à long terme**, ce qui peut limiter ses performances dans certaines tâches linguistiques complexes. Des variantes, comme les **HMM d’ordre supérieur**, permettent d’intégrer plus de contexte mais augmentent aussi le coût computationnel.  

Aujourd’hui, les modèles probabilistes comme les HMM ont été en grande partie remplacés par des approches plus avancées en **traitement automatique du langage naturel (TALN)**. Des modèles comme les **CRF (Conditional Random Fields)** offrent une alternative plus flexible en modélisant directement la distribution des séquences complètes, tandis que les **réseaux neuronaux récurrents (LSTM, GRU)** et les **modèles Transformers (BERT, GPT)** exploitent de vastes quantités de données et capturent des relations complexes entre mots, améliorant ainsi considérablement la modélisation du langage.  

Enfin, les **langues morphologiquement riches**, comme le finnois ou l’arabe, posent un défi particulier aux HMM classiques, qui ne tiennent pas compte de la structure interne des mots. L’intégration d’informations morphologiques et syntaxiques devient alors essentielle pour obtenir une analyse plus fine des séquences.  

Bien que les HMM aient des limites évidentes, ils restent une **base théorique précieuse** pour comprendre la modélisation des séquences et ont servi de fondation à de nombreuses avancées en TAL.